In [28]:
import pandas as pd
import numpy as np
import sklearn as sk
import sklearn.datasets
import matplotlib.pyplot as plt
from keras.datasets import mnist

In [29]:
#Softmax Function
def softmax(u):
    return np.exp(u) / np.sum(np.exp(u),axis=0)

In [30]:
#Cross-Entropy Loss Function 
def multiclass_cross_entropy(p,q):
    return -np.vdot(p,np.log(q))

In [31]:
#Objective Function
def L(beta,X,y):
    N = X.shape[0]
    s = 0
    for i in range(N):
        xiHat = X[i]
        yi = y[i]
        
        #Prediction Function
        u = beta @ xiHat
        yi_pred = softmax(u)
        
        #Calculating the loss
        s += multiclass_cross_entropy(yi,yi_pred)
    
    #Returns the average loss 
    return s / N 

In [32]:
#Function to perform Multiclass Logistic Regression using Stochastic Gradient Descent
def multiLogReg_SGD(X,y,batch,a_val,ep): #(Training, Testing, Batch_Size, Alpha, Epoch)
    num_epochs = ep
    alpha = a_val
    N, d = X.shape
    X = np.insert(X,0,1,axis=1) #Inserting Leading 1's to create augmented matrix
    K = y.shape[1] #Setting K to the number of classes in the test set
    
    batch_size = batch #Initialising batch_size 
    
    beta = np.zeros((K,d+1)) #Initialising beta vector
    L_vals = []
    
    for epoch in range(num_epochs):
        L_val = L(beta,X,y) #Calculate the loss for every epoch
        L_vals.append(L_val) #Add the losses into L_vals
        
        print('| Epoch : ' + str(epoch) + ' | Loss : ' + str(L_val))
        
        prm = np.random.permutation(N) #A random permutation of size N 
        
        batch_idx = 0
        for start_idx in range(0,N,batch_size): #Starting from 0, to length N in steps of the batch size
            
            stop_idx = start_idx + batch_size 
            stop_idx = min(stop_idx, N)
            
            num_examples_in_batch = stop_idx - start_idx 
        
            for i in prm[start_idx:stop_idx]:
                xiHat = X[i]
                yi = y[i]
                u = beta @ xiHat
                
                #Prediction Function
                yi_pred = softmax(u)
                
                #Calculating gradient 
                grad = np.outer(yi_pred - yi, xiHat)

            #Divide gradient by number of examples in batch 
            grad = grad / num_examples_in_batch
            beta = beta - alpha*grad 
            
            batch_idx += 1
            
    
    return beta, L_vals
    

In [33]:
#Function to predict labels
def predictLabels(beta,X):
    X = np.insert(X,0,1,axis = 1)
    N = X.shape[0]
    
    predictions = []
    
    for i in range(N):
        xiHat = X[i]
        yi_pred = softmax(beta @ xiHat)
        
        k = np.argmax(yi_pred)
        predictions.append(k)
        
    return predictions

In [34]:
#Function to calculate accuracy
def accuracy_score(predicted_labels,truth_labels):
    
    yi_pred = np.array(predicted_labels)
    truth = np.array(truth_labels)
    num_correct = sum(p == t for p, t in zip(yi_pred, truth)) 
    accuracy = num_correct/truth_labels.shape[0]
    
    return accuracy

In [35]:
#Loading Dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [36]:
X_train = X_train/255
X_test = X_test/255

In [37]:
#Reshaping the training set 
N_train, numRows, numCols = X_train.shape
X_train = np.reshape(X_train,(N_train,numRows*numCols))

In [38]:
#One-Hot Encoding
y_train = pd.get_dummies(y_train).values

In [ ]:
beta, L_vals = multiLogReg_SGD(X_train,y_train,10,0.01,100)

| Epoch : 0 | Loss : 2.3025850929954172
| Epoch : 1 | Loss : 0.7478375617837253
| Epoch : 2 | Loss : 0.575040907389319
| Epoch : 3 | Loss : 0.507214506652957
| Epoch : 4 | Loss : 0.4688793489334052
| Epoch : 5 | Loss : 0.44434912608452726
| Epoch : 6 | Loss : 0.42621315180443736
| Epoch : 7 | Loss : 0.4109275839995608
| Epoch : 8 | Loss : 0.39901411138491233
| Epoch : 9 | Loss : 0.3911669494263595
| Epoch : 10 | Loss : 0.38406908033427917
| Epoch : 11 | Loss : 0.3747312106627982
| Epoch : 12 | Loss : 0.3693341036614135
| Epoch : 13 | Loss : 0.3636067732590776
| Epoch : 14 | Loss : 0.35898253871933455
| Epoch : 15 | Loss : 0.35554483817212135
| Epoch : 16 | Loss : 0.3525378087765864
| Epoch : 17 | Loss : 0.3485334629427758
| Epoch : 18 | Loss : 0.3444823664924098
| Epoch : 19 | Loss : 0.3413492470289609
| Epoch : 20 | Loss : 0.3386846220285912
| Epoch : 21 | Loss : 0.33732613891612
| Epoch : 22 | Loss : 0.3343641516373868
| Epoch : 23 | Loss : 0.3348793184604953
| Epoch : 24 | Loss : 0.

In [ ]:
plt.plot(L_vals)

In [ ]:
#Reshaping the test array
N_test = X_test.shape[0]
X_test = np.reshape(X_test,(N_test,numRows*numCols))

In [ ]:
#Get predictions for X_test
predictions = predictLabels(beta,X_test)

In [ ]:
#First 10 rows of the predictions
predictions[0:10]

In [ ]:
#First 10 rows of the truth
y_test[0:10]

In [ ]:
#Accuracy : 
accuracy_score(predictions,y_test)